# 집계 연산
- 집계 : 무언가를 함께 모으는 행위이며 빅데이터 분석의 초석
- 집계 함수 : 집계를 수행하려면 키나 그룹을 지정하고 하나 이상의 컬럼을 변환하는 방법을 지정할 수 있음
  - 여러 입력값이 주어지면 그룹별로 결과를 생성함
- 스파크의 집계 능력은 다양한 활용 사례와 가능성으로 비추어보았을 때 매우 정교하며 충분히 발달해 있음
  - 일반적으로 특정 그룹의 평균값을 구하는 것과 같은 수치형 데이터 요약에 집계를 사용할 수 있음
  - 해당 연산은 합산, 곱셈 또는 카운팅 등의 작업이 될 수 있음
  - 배열, 리스트 또는 맵 같은 복합 데이터 타입을 사용해 집계를 수행할 수도 있음
  
- 스파크는 모든 데이터 타입을 다루는 것 외에도 다음과 같은 그룹화 데이터 타입을 생성할 수 있음
  - 가장 간단한 형태의 그룹화 : select 구문에서 집계를 수행해 DataFrame의 전체 데이터를 요약하는 것
  - 'group by' : 하나 이상의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용할 수 있음
  - '윈도우(window)' : 하나 이상의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용할 수 있음.
    - 하지만 함수의 입력으로 사용할 로우는 현재 로우와 어느 정도 연관성이 있어야 함
  - 그룹화 셋(grouping set) : 서로 다른 레벨의 값을 집계할 때 사용함
    - SQL, DataFrame의 롤업, 큐브를 사용할 수 있음
  - 롤업(rollup) : 하나 이상의 키를 지정할 수 있음
    - 컬럼을 변환하는 데 다른 집계 함수를 사용하여 계층적으로 요약된 값을 구할 수 있음
  - 큐브(cube) : 하나 이상의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용할 수 있음
    - 모든 컬럼 조합에 대한 요약 값을 계산함
- 지정된 집계 함수에 따라 그룹화된 결과 : RelationalGroupedDataset을 반환

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("sample").master("local[*]").getOrCreate()

In [2]:
# 구매 이력 데이터를 사용해 파티션을 훨씬 적은 수로 분할할 수 있도록 리파티셔닝하고 빠르게 접근할 수 있도록 캐싱하겠음
# 파티션 수를 줄이는 이유 : 적은 양의 데이터를 가진 수많은 파일이 존재하기 때문
df=spark.read.format("csv")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("./Spark-The-Definitive-Guide-master/data/retail-data/all/online-retail-dataset.csv")\
    .coalesce(5)
df.cache()
df.createOrReplaceTempView("dfTable")

In [3]:
#DataFrame을 사용해 기본 집계를 수행해보겠음
#다음은 count 메서드를 사용한 간단한 예제임
df.count()==541909

True

- count 메서드가 트랜스포메이셔이 아닌 액션잉라는 사실을 알고 있을 것임(앞에서 배움)
  - 그러므로 결과를 즉시 반환함
  - count 메서드는 데이터셋의 전체 크기를 알아보는 용도로 사용하지만 메모리에 DataFrame 캐싱 작업을 수행하는 용도로 사용되기도 함
  - count 메서드가 약간 이질적으로 보일 수 있음
    - why : 함수가 아니라 메서드 형태로 존재하고 트랜스포메이션처럼 지연 연산 방식이 아닌 즉시 연산을 수행하기 때문임
    - 다음 절에서는 지연 연산 방식으로 count 메서드를 사용하는 방법을 알아보겠음

## 7.1 집계 함수
- 모든 집계는 DataFrame의 .stat 속성을 이용하는 특별한 경우를 제외한다면 함수를 사용함

### 7.1.1 count
- count 함수 동작 방식
  - 액션 x
  - 트랜스포메이션
- count 함수 사용 방법(2가지)
  - 하나, count 함수에 특정 컬럼을 지정하는 방식
  - 둘, count(\*)나 count(\1)을 사용하는 방식

In [4]:
#count 함수를 사용해 전체 로우 수를 카운트할 수 있음
from pyspark.sql.functions import count
df.select(count("StockCode")).show()
# -> sql
#SELECT COUNT(*) FROM dfTable
#-> count(*) 구문을 사용하면 null 값을 가진 로우를 포함해 카운트함
#count 함수에 특정 컬럼을 지정하면 null 값을 카운트하지 않음

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



### 7.1.2 countDistinct
- countDistrinct 함수 : 전체 레코드 수가 아닌 고유 레코드 수를 구할 수 있음
  - 개별 컬럼을 처리하는 데 더 적합함

In [5]:
from pyspark.sql.functions import countDistinct

df.select(countDistinct("StockCode")).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



### 7.1.3 approx_count_distinct
- approx_count_distinct
  - 어느 정도 수준의 정확도를 가지는 근사치만으로도 유의미할 때 사용
  - 근사치 계산 가능
  - 최대 추정 오류율이라는 한 가지 파라미터를 더 사용함
  - countDistinct 함수보다 더 빠르게 결과를 반환함
  - 대규모 데이터셋을 사용할 때 훨씬 더 좋아짐

In [6]:
#큰 오류율 설정했기 때문에 크게 벗어나는 결과를 얻게 됨
from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("StockCode",0.1)).show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



### 7.1.4 first와 last
- first : DataFrame의 첫 번째 값
- last : DataFrame의 마지막 값
  - 위의 함수들은 값이 아닌 로우를 기반으로 동작함

In [7]:
from pyspark.sql.functions import first, last

df.select(first("StockCode"),last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



### 7.1.5 min과 max
- min : 최솟값
- max : 최댓값

In [8]:
from pyspark.sql.functions import min,max
df.select(min("Quantity"),max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



### 7.1.6 sum
- sum
  - DataFrame에서 특정 컬럼의 모든 값 합산

In [9]:
from pyspark.sql.functions import sum
df.select(sum("Quantity")).show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



### 7.1.7 sumDistinct
- sumDistinct
  - 특정 컬럼의 모든 값을 합산하는 방법 외에도 고윳값을 합산할 수 있음

In [10]:
from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("Quantity")).show()

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



### 7.1.8 avg
- avg
  - sum 함수의 결과를 count 함수의 결과로 나누어 평균값을 구할 수 있음
  - 스파크의 avg 함수나 mean 함수를 사용하면 평균값을 더 쉽게 구할 수 있음

In [11]:
#집계된 컬렁믈 재활용하기 위해 alias 메서드를 사용함
from pyspark.sql.functions import sum,count,avg,expr
df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"))\
    .selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases").show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



### 7.1.9 분산과 표준편차
- 분산과 표준편차: 평균 주변에 데이터가 분포된 정도를 측정하는 방법
  - 분산 : 평균과의 차이를 제곱한 결과의 평균
  - 표준편차 : 분산의 제곱근
- 스파크에서 분산과 표준편차 계산 가능
  - 표본표준편차뿐만 아니라 모표준편차 방식도 지원하기 때문에 주의가 필요함
    - variance : 표본표준분산
    - stddev : 표본표준편차
    - var_pop : 모표준분산
    - stddev_pop : 모표준편차

In [12]:
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp
df.select(var_pop("Quantity"),var_samp("Quantity"),
         stddev_pop("Quantity"),stddev_samp("Quantity")).show()

+------------------+------------------+--------------------+---------------------+
| var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
|47559.303646609056|47559.391409298754|  218.08095663447796|   218.08115785023418|
+------------------+------------------+--------------------+---------------------+



### 7.1.10 비대칭도와 첨도
- 비대칭도와 첨도 : 모두 데이터의 변곡점을 측정하는 방법
  - 확률변수와 화귤분포로 데이터를 모델링할 때 특히 중요함
- 비대칭도 : 데이터 평균의 비대칭 정도를 측정
- 첨도 : 데이터 끝 부분을 측정함

In [13]:
from pyspark.sql.functions import skewness,kurtosis
df.select(skewness("Quantity"),kurtosis("Quantity")).show()

+-------------------+------------------+
| skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
|-0.2640755761052562|119768.05495536952|
+-------------------+------------------+



### 7.1.11 공분산과 상관관계
- 두 컬럼값 사이의 영향도를 비교하는 함수
- cov : 공분산
  - 데이터 입력값에 따라 다른 범위를 가짐
- corr : 상관관계
  - 피어슨 상관계수를 측정함
  - -1과 1 사이의 값을 가짐
  - 모집단이나 표본에 대한 계산 개념이 없음
- var 함수처럼 표본공분산 방식이나 모공분산 방식으로 공분산을 계산할 수도 있음
  - 사용하고자 하는 방식을 명확하게 지정하는 것이 좋음

In [14]:
from pyspark.sql.functions import corr,covar_pop,covar_samp

df.select(corr("InvoiceNo","Quantity"),covar_samp("InvoiceNo","Quantity"),
         covar_pop("InvoiceNo","Quantity")).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085635685E-4|             1052.7280543902734|            1052.7260778741693|
+-------------------------+-------------------------------+------------------------------+



### 7.1.12 복합 데이터 타입의 집계
- 스파크는 수식을 이용한 집계뿐만 아니라 복합 데이터 타입을 사용해 집계를 수행할 수 있음
  - ex) 특정 컬럼의 값을 리스트로 수집하거나 셋 데이터 타입으로 고윳값만 수집할 수 있음
- 수집된 데이터는 처리 파이프라인에서 다양한 프로그래밍 방식으로 다루거나 사용자 정의 함수를 사용해 전체 데이터에 접근할 수 있음

In [15]:
from pyspark.sql.functions import collect_set,collect_list
df.agg(collect_set("Country"),collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



## 7.2 그룹화
- 데이터 그룹 기반의 집계를 수행하는 경우
  - 단일 컬럼의 데이터를 그룹화하고 해당 그룹의 다른 여러 컬럼을 사용해서 계산하기 위해 카테고리형 데이터를 사용함
- 데이터 그룹 기반의 집계를 설명하는 데 가장 좋은 방법 : 그룹화
  - 이전에 했던 것처럼 카운트를 가장 먼저 수행함
    - 해당 연산은 또 다른 DataFrame을 반환하여 지연 처리 방식으로 수행됨
- 그룹화 작업 두 단계로 이루어짐
  - 1. 하나 이상의 컬럼을 그룹화 : RelationalGroupedDataset이 반환
  - 2. 집계 연산을 수행 : DataFrame이 반환됨
  
- 그룹의 기준이 되는 컬럼을 여러 개 지정할 수 있음

In [16]:
df.groupBy("InvoiceNo","CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
|   538800|     16458|   10|
|   538942|     17346|   12|
|  C539947|     13854|    1|
|   540096|     13253|   16|
|   540530|     14755|   27|
|   541225|     14099|   19|
|   541978|     13551|    4|
|   542093|     17677|   16|
|   536596|      null|    6|
|   537252|      null|    1|
|   538041|      null|    1|
|   543188|     12567|   63|
|   543590|     17377|   19|
|  C543757|     13115|    1|
|  C544318|     12989|    1|
+---------+----------+-----+
only showing top 20 rows



### 7.2.1 표현식을 이용한 그룹화
- 카운팅
  - 메서드로 사용할 수 있으므로 조금 특별함
  - 메서드 대신 count 함수를 사용할 것을 추천함
  - count 함수를 select 구문에 표현식으로 지정하는 것보다 agg 메서드를 사용하는 것이 좋음
- agg 메서드
  - 여러 집계 처리를 한 번에 지정할 수 있으며 집계에 표현식을 사용할 수 있음
  - 트랜스포메이션이 완료된 컬럼에 alias 메서드를 사용할 수 있음

In [18]:
from pyspark.sql.functions import count
df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)")).show(2)

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
+---------+----+---------------+
only showing top 2 rows



### 7.2.2 맵을 이용한 그룹화
- 컬럼을 키로, 수행할 집계 함수의 문자열을 값으로 하는 맵 타입을 사용해 트랜스포메이션을 정의할 수 있음
- 수행할 집계 함수를 한 줄로 작성하면 여러 컬럼명을 재사용할 수 있음

In [19]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)")).show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1

## 7.3 윈도우 함수
- 윈도우 함수
  - 집계를 사용할 수도 있음
  - 데이터의 특정 윈도우를 대상으로 고유의 집계 연산을 수행함
  - 데이터의 '윈도우' : 현재 데이터에 대한 참조를 사용해 정의함
  - 윈도우 명세 : 함수에 전달될 로우를 결정함
  - group-by 함수와 유사해보일 수도 있으나 둘의 차이점이 존재

- group-by 함수를 사용하면 모든 로우 레코드가 단일 그룹으로만 이동함